In [1]:
from transformers import RobertaForMaskedLM, RobertaTokenizer
import torch
from tqdm import tqdm

In [35]:
model = RobertaForMaskedLM.from_pretrained('./output/')
tokenizer = RobertaTokenizer.from_pretrained('./output/')

In [3]:
import os
import pandas as pd
from random import choice

import nltk

In [4]:
path_to_intents = os.path.join('data', 'raw')
intents = os.listdir(path_to_intents)
get_path = lambda x: os.path.join('data', 'raw', x, x + ".csv")

In [5]:
os.chdir('helpers')
from analyze import questions, entities, get_data
data = get_data()
os.chdir('..')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dhruv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
from collections import defaultdict
text = defaultdict(list)
encoded_text = defaultdict(list)

ans = defaultdict(list)
response = defaultdict(list)

scores = defaultdict(list)

for (_questions, intent, entity) in zip(questions, intents, entities):
    question = choice(_questions)
    
    for row in data[intent]['df']['text']:
        text[intent].append("<s> " + row.strip() + '. ' + question.strip() + "<mask> <mask> <mask> </s>")
        
    for row in data[intent]['df'][entity]:
        ans[intent].append((str(row)).split())

In [21]:
for intent in intents:
    for row in text[intent]:
        encoded_text[intent].append(torch.tensor([tokenizer.encode(row, add_special_tokens=False)]))

In [22]:
encoded_text['BookRestaurant'][0]

tensor([[    0,  6298,    20,  2367,   953,  1437,  2391,    11,  2808,    13,
          5996,     4,  6834, 19969,   219,   116, 50264, 50264, 50264,     2]])

In [23]:
text['BookRestaurant'][0]

'<s> book The Middle East  restaurant in IN for noon. Which eatery?<mask> <mask> <mask> </s>'

In [24]:
ans['BookRestaurant'][0]

['The', 'Middle', 'East', 'in', 'IN']

In [25]:
with torch.no_grad():
    for intent in intents:
        for i, row in tqdm(enumerate(encoded_text[intent])):
            out = model(row)
            actual_out = model((torch.argmax(out[0][0], dim=1)).view(1, -1))
#             print(actual_out)
            response[intent].append(tokenizer.decode(torch.argmax(out[0][0], dim=1).tolist()).split())
            print(response[intent][-1])
            score = nltk.translate.bleu_score.sentence_bleu(
                    [ans[intent][i]],
                    response[intent][i],
                    smoothing_function=nltk.translate.bleu_score.SmoothingFunction().method4,
                    auto_reweigh=True
                )
            



0it [00:00, ?it/s]

1it [00:00,  3.85it/s]



['<s>Add', 'another', 'song', 'to', 'the', 'Cita', 'RomÃ¡ntica', 'playlist..', 'Which', 'playlist?</s>??</s>']
['<s>add', 'clem', 'burke', 'in', 'my', 'playlist', 'Pre-Party', 'R&B', 'Jams</s>', 'Which', 'playlist?</s>??</s>']


2it [00:00,  4.22it/s]

3it [00:00,  4.61it/s]



['<s>Add', 'Live', 'from', 'Aragon', 'Ballroom', 'to', 'Trapeo</s>', 'Which', 'playlist?</s>', 'tuned.</s>']
['<s>add', 'Unite', 'and', 'Win', 'to', 'my', 'next', 'night', 'out</s>', 'Which', 'playlist?</s>', 'playlist?</s>']


4it [00:00,  4.79it/s]

5it [00:01,  4.15it/s]

...


['<s>Add', 'a', 'tune', 'by', 'Louis', 'Wolfe', 'Gilbert', 'to', 'my', '_____', 'playlist</s>', 'Which', 'playlist?</s>', 'favorite.</s>']



421it [01:24,  4.70it/s]



422it [01:24,  4.80it/s]

423it [01:24,  5.17it/s]

['<s>Please', 'add', 'a', 'tune', 'by', 'kaori', 'utatsuki', 'to', 'my', 'orean', 'OSTs', 'playlist</s>', 'Which', 'playlist?</s>::</s>']
['<s>Add', 'it', 'to', 'my', 'playlist', 'Classic', 'Punk.', 'Which', 'playlist?</s>?.</s>']


KeyboardInterrupt: 

In [ ]:
" ".join(response['BookRestaurant'][0])

In [ ]:
for intent in intents:
    print(f"For {intent}, avg. BLEU score is {sum(scores[intent]) / len(scores[intent])}")

In [ ]:
os.chdir('transformers')
from examples.run_generation import sample_sequence
os.chdir('..')
_out = sample_sequence(
    model, 5, encoded_text['BookRestaurant'][0][0], top_p = 0.9,
    is_xlm_mlm=True, xlm_mask_token=50264
)

In [ ]:
encoded_text['BookRestaurant'][0][0]

In [ ]:
tokenizer

In [ ]:
tokenizer.decode(_out.squeeze(0).tolist())

In [ ]:
tokenizer.decode([50264])

In [ ]:
_out.squeeze(0).tolist()

In [ ]:
scores

In [ ]:
response

In [ ]:
def fill_mask(masked_input, model, tokenizer, topk=5):
    # Adapted from https://github.com/pytorch/fairseq/blob/master/fairseq/models/roberta/hub_interface.py
    assert masked_input.count('<mask>') == 1
    input_ids = torch.tensor(tokenizer.encode(masked_input, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
    logits = model(input_ids)[0]  # The last hidden-state is the first element of the output tuple
    masked_index = (input_ids.squeeze() == tokenizer.mask_token_id).nonzero().item()
    logits = logits[0, masked_index, :]
    prob = logits.softmax(dim=0)
    values, indices = prob.topk(k=topk, dim=0)
    topk_predicted_token_bpe = ' '.join([tokenizer.convert_ids_to_tokens(indices[i].item())
                                         for i in range(len(indices))])
    masked_token = tokenizer.mask_token
    topk_filled_outputs = []
    for index, predicted_token_bpe in enumerate(topk_predicted_token_bpe.split(' ')):
        predicted_token = predicted_token_bpe.replace('\u2581', ' ')
        if " {0}".format(masked_token) in masked_input:
            topk_filled_outputs.append((
                masked_input.replace(
                    ' {0}'.format(masked_token), predicted_token
                ),
                values[index].item(),
                predicted_token,
            ))
        else:
            topk_filled_outputs.append((
                masked_input.replace(masked_token, predicted_token),
                values[index].item(),
                predicted_token,
            ))
    return topk_filled_outputs


In [36]:
with open("data/masked.txt", 'r', encoding='utf-8') as f:
    for x in f.read().split("\n\n"):
        _x = tokenizer.encode(x, add_special_tokens=False)
        out = model(torch.tensor([_x]))
        print(" ".join(tokenizer.decode(torch.argmax(out[0][0], dim=1).tolist()).split()))
#         print(fill_mask("The answer is <mask>.", model, tokenizer, topk=1)[0])

<s>add Top 100 Indie Tracks on Spotify to my trackWhich playlist? mymy track Indie Indie track
<s>add Top 100 Indie Tracks on Spotify to my track Where should I add? my track track to Indie track
<s>add Top 100 Indie Tracks on Spotify to my track What should I add to? my Top 100 Indie Indie track
<s>add Top 100 Indie Tracks on Spotify to my track What was the playlist? my track track to Indie track
<s>add this clint mansell song to latin pop classicsWhich playlist? latlatin
<s>add this clint mansell song to latin pop classics Where should I add? latlatin
<s>add this clint mansell song to latin pop classics What should I add to? latlatin
<s>add this clint mansell song to latin pop classics What was the playlist? latlatin
<s>Add Fair Charlotte to the We Everywhere playlist Which playlist? WeWe
<s>Add Fair Charlotte to the We Everywhere playlist Where should I add? toWe
<s>Add Fair Charlotte to the We Everywhere playlist What should I add to? WeWe
<s>Add Fair Charlotte to the We Everywher

<s>I want a matt brown tune in my Fresh Finds Fire Emoji playlist Which playlist? my Fresh Find Fire
<s>I want a matt brown tune in my Fresh Finds Fire Emoji playlist Where should I add? tomy Fresh Find Fire
<s>I want a matt brown tune in my Fresh Finds Fire Emoji playlist What should I add to? mymy Fresh Find Fire
<s>I want a matt brown tune in my Fresh Finds Fire Emoji playlist What was the playlist? mymy Fresh Find Fire
<s>I want to put this track on diann's Just Smile playlist Which playlist? dJust Smile
<s>I want to put this track on diann's Just Smile playlist Where should I add? dJust Smile
<s>I want to put this track on diann's Just Smile playlist What should I add to? dJust Smile
<s>I want to put this track on diann's Just Smile playlist What was the playlist? dJust Smile
<s>add this track to polka's dance playlist Which playlist? hermy
<s>add this track to polka's dance playlist Where should I add? tomy
<s>add this track to polka's dance playlist What should I add to? hers
<s

KeyboardInterrupt: 

In [29]:
_x

[0,
 4917,
 3107,
 727,
 35890,
 41902,
 15,
 13703,
 7,
 127,
 1437,
 1349,
 6834,
 30475,
 116,
 50264,
 50264,
 50264,
 50264,
 50264,
 50264,
 50264,
 2]